In [1]:
import nltk
import pandas as pd
import re
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer
import pymysql

In [2]:
def read_mysql_table(table, host='localhost', user='root', password='', database='review'):
    # Establish a connection to the MySQL database
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    
    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()
    
    query = f"SELECT * FROM {table}"
    cursor.execute(query)
    result = cursor.fetchall()
    
    # Convert the result to a Pandas DataFrame
    df = pd.DataFrame(result)
    
    # Assign column names based on the cursor description
    df.columns = [column[0] for column in cursor.description]
    
    # Close the cursor and the database connection
    cursor.close()
    connection.close()
    
    return df

In [3]:
table_name = 'input_review'
df = read_mysql_table(table_name)
df.head()

,id_review,nama,tanggal,review
0,13,User1,2023-12-04,Aplikasi ini sangat membantu dan mudah digunakan.
1,14,User2,2023-12-04,Website ini memberikan pengalaman yang luar bi...
2,15,User3,2023-12-04,Saya sangat puas dengan fitur-fitur yang dised...
3,16,User4,2023-12-04,Layanan pelanggan yang ramah dan responsif.
4,17,User5,2023-12-04,Aplikasi ini membuat hidup saya lebih mudah da...


In [4]:
#menyimpan tweet. (tipe data series pandas)
data_content = df['review']

In [5]:
# casefolding
data_casefolding = data_content.str.lower()
data_casefolding.head()

0    aplikasi ini sangat membantu dan mudah digunakan.
1    website ini memberikan pengalaman yang luar bi...
2    saya sangat puas dengan fitur-fitur yang dised...
3          layanan pelanggan yang ramah dan responsif.
4    aplikasi ini membuat hidup saya lebih mudah da...
Name: review, dtype: object

In [6]:
#filtering

#url
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", str(tweet)) for tweet in data_casefolding]
#cont
filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
#punctuatuion
filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]
#  hapus #tagger
filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
#numeric
filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]

# # filtering RT , @ dan #
# fungsi_clen_rt = lambda x: re.compile('\#').sub('', re.compile('rt @').sub('@', x, count=1).strip())
# clean = [fungsi_clen_rt for tweet in filtering_numeric]

data_filtering = pd.Series(filtering_numeric)


In [7]:
# #tokenize
tknzr = TweetTokenizer()
data_tokenize = [tknzr.tokenize(tweet) for tweet in data_filtering]
data_tokenize

[['aplikasi', 'ini', 'sangat', 'membantu', 'dan', 'mudah', 'digunakan'],
 ['website', 'ini', 'memberikan', 'pengalaman', 'yang', 'luar', 'biasa'],
 ['saya', 'sangat', 'puas', 'dengan', 'fitur', 'fitur', 'yang', 'disediakan'],
 ['layanan', 'pelanggan', 'yang', 'ramah', 'dan', 'responsif'],
 ['aplikasi',
  'ini',
  'membuat',
  'hidup',
  'saya',
  'lebih',
  'mudah',
  'dan',
  'efisien'],
 ['banyak', 'bug', 'pada', 'aplikasi', 'membuat', 'pengalaman', 'buruk'],
 ['navigasi', 'website', 'sangat', 'membingungkan', 'dan', 'lambat'],
 ['fitur', 'fitur', 'yang', 'ditawarkan', 'kurang', 'memuaskan'],
 ['aplikasi', 'sering', 'crash', 'dan', 'tidak', 'stabil'],
 ['pelayanan', 'pelanggan', 'sangat', 'buruk', 'dan', 'tidak', 'responsif'],
 ['saya', 'merasa', 'biasa', 'saja', 'dengan', 'aplikasi', 'ini'],
 ['website',
  'ini',
  'memiliki',
  'kelebihan',
  'dan',
  'kekurangan',
  'masing',
  'masing'],
 ['tidak',
  'terlalu',
  'terkesan',
  'dengan',
  'fitur',
  'fitur',
  'yang',
  'disediak

In [8]:
#slang word
path_dataslang = open("Data/kamus kata baku-clear.csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

data_formal = []
for data in data_tokenize:
  data_clean = [replaceSlang(word) for word in data]
  data_formal.append(data_clean)
len_data_formal = len(data_formal)
# print(data_formal)
# len_data_formal

In [9]:
nltk.download('stopwords')
default_stop_words = nltk.corpus.stopwords.words('indonesian')
stopwords = set(default_stop_words)

def removeStopWords(line, stopwords):
  words = []
  for word in line:  
    word=str(word)
    word = word.strip()
    if word not in stopwords and word != "" and word != "&":
      words.append(word)

  return words
reviews = [removeStopWords(line,stopwords) for line in data_formal]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# Specify the file path of the pickle file
file_path = 'Model/training.pickle'

# Read the pickle file
with open(file_path, 'rb') as file:
    data_train = pickle.load(file)

In [11]:
data_train[0]

'ganjarpranowo malu dukung kualitas dukung gantung dukung dukung tolol dukung'

In [12]:
# pembuatan vector kata
vectorizer = TfidfVectorizer()
train_vector = vectorizer.fit_transform(data_train)
reviews2 = [" ".join(r) for r in reviews]

In [13]:
load_model = pickle.load(open('Model/tfidf_Model Naive Bayes_nvb.pkl','rb'))

result = []

for test in reviews2:
    test_data = [str(test)]
    test_vector = vectorizer.transform(test_data)
    pred = load_model.predict(test_vector)
    result.append(pred[0])

In [14]:
result

[1, -1, 0, 1, 1, 0, -1, -1, -1, 0, -1, -1, -1, 0, -1]

In [15]:
from sklearn.utils.multiclass import unique_labels
unique_labels(result)

array([-1,  0,  1], dtype=int64)

In [16]:
df['label'] = result

In [17]:
def delete_all_data_from_table(table, host='localhost', user='root', password='', database='review'):
    # Establish a connection to the MySQL database
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    
    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()
    
    # Delete all data from the specified table
    query = f"DELETE FROM {table}"
    cursor.execute(query)
    
    # Commit the changes
    connection.commit()
    
    # Close the cursor and the database connection
    cursor.close()
    connection.close()

In [18]:
delete_all_data_from_table('input_review')

In [19]:
def insert_df_into_hasil_model(df, host='localhost', user='root', password='', database='review'):
    # Establish a connection to the MySQL database
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )

    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()

    # Insert each row from the DataFrame into the 'hasil_model' table
    for index, row in df.iterrows():
        query = "INSERT INTO hasil_model (id_review, nama, tanggal, review, label) VALUES (%s, %s, %s, %s, %s)"
        cursor.execute(query, (row['id_review'], row['nama'], row['tanggal'], row['review'], row['label']))

    # Commit the changes
    connection.commit()

    # Close the cursor and the database connection
    cursor.close()
    connection.close()

In [20]:
insert_df_into_hasil_model(df)

In [22]:
table_name = 'hasil_model'
hasil_df = read_mysql_table(table_name)
hasil_df.to_csv('Data/hasil_model.csv')